    # Takes in input from the user and prints a (Filler) response
* If the input is exit, quit, goodbye or bye in any case sensitive way, the program must print:

"A: Goodbye"
and exit

In [ ]:
from supervised_learning.qa_bot.main import reference

while True:    
    A = input("Q: ")

    exit = ["exit", "quit", "goodbye", "bye"]

    if A.lower() in exit:
        print("A: Goodbye")
        break
    else:   
        print("A:")
        


### That was the first task of this variety.
* For the second one we take the previous one and basically use the reference text to answer the questions.
- If the answer is not to be found: "Sorry, I do not understand your question"

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer

In [ ]:
def base():
    while True:    
        global A
        A = input("Q: ")
        exit = ["exit", "quit", "goodbye", "bye"]

        if A.lower() in exit:
            print("A: Goodbye")
            break
        else:   
            answer_loop(reference)


def answer_loop(reference):
    """
    Answers questions based on a reference text
    
    Parameters:
        reference -> reference text given
    
    Returns:
        the answer or "Sorry I do not understand your question"
    """
    bert = hub.load("https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-l-12-h-768-a-12/4")
    tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
    
    # Tokenizes the questions and references
    tokens_q = tokenizer(A, return_tensors="tf", truncation=True, padding=True)["input_ids"]
    tokens_r = tokenizer(reference, return_tensors="tf", truncation=True, padding=True)["input_ids"]
    
    # Combine the questiona and reference documents into one
    combine = tf.concat([tokens_q[:, : 1024], tokens_r[:, 1024 :]], axis=-1)
    
    # Make the predictions
    outputs = bert(combine)[0]
    logits = tf.argmax(outputs, axis=1).numpy()
    
    # Get the answer based on predicted idx
    if logits[0] < len(tokens_q[0]):
        return tokenizer.decode([tokens_r[0][logits[0]]])
    else:
        return None
    